In [1]:

%load_ext autoreload
%autoreload 2


In [2]:

import numpy as np
import pickle
import matplotlib.pyplot as plt

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


#RUN BEFORE#

In [3]:
#RUN AFTER#

import tensorflow.keras as keras
K = keras.backend
from util.Models import *
from util.Generators import *


In [4]:

data_path = '/fast_scratch_1/atlas_images/v01-45/'

cell_geo_path = data_path + 'cell_geo.root'

out_path = '/home/joshualerner/start_tf/PionReconstruction/data/'


In [5]:

K.clear_session()
model = GarNetModel()


2023-05-17 17:05:33.264247: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2023-05-17 17:05:33.264316: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9671 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1b:00.0, compute capability: 7.5


Model: "gar_net_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 4)]     0                                            
__________________________________________________________________________________________________
input_gex (GlobalExchange)      (None, 128, 8)       0           input_1[0][0]                    
__________________________________________________________________________________________________
input_batchnorm (BatchNormaliza (None, 128, 8)       32          input_gex[0][0]                  
__________________________________________________________________________________________________
input_dense (Dense)             (None, 128, 8)       72          input_batchnorm[0][0]            
______________________________________________________________________________________

In [6]:

train_val_split = 0.8
batch_size = 64

pi0_list = [[data_path + f'pi0/user.angerami.24559740.OutputStream._0000{i}.root', 1] for i in range(11, 21)]
pipm_list = [[data_path + f'pipm/user.angerami.24559744.OutputStream._0000{i}.root', 0] for i in range(11, 21)]

train_start = 0
train_end = train_start + int(train_val_split*len(pi0_list))
val_start = train_end
val_end = len(pi0_list)
train_file_list = np.concatenate((pi0_list[train_start:train_end], pipm_list[train_start:train_end]))
val_file_list = np.concatenate((pi0_list[val_start:val_end], pipm_list[val_start:val_end]))


In [9]:

train_generator = garnetDataGenerator(train_file_list, 
                                      cell_geo_path, 
                                      batch_size, 
                                      labeled=True, 
                                      preprocess=False, 
                                      output_dir=out_path + 'train/')

validation_generator = garnetDataGenerator(val_file_list, 
                                           cell_geo_path, 
                                           batch_size, 
                                           labeled=True, 
                                           preprocess=False, 
                                           output_dir=out_path + 'val/')



Loading Geo Dictionary...
	Starting on cell_geo_sampling
	Starting on cell_geo_eta
	Starting on cell_geo_phi
Finished loading Geo Dictionary

Loading Geo Dictionary...
	Starting on cell_geo_sampling
	Starting on cell_geo_eta
	Starting on cell_geo_phi
Finished loading Geo Dictionary


In [15]:

history = model.fit(train_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=train_generator.num_procs,
                    shuffle=True,
                    verbose=1)

model.save(out_path + 'models/GarNet/')

with open(out_path + 'models/GarNet/history.pickle', 'wb') as handle:
    pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)

ValueError: Failed to find data adapter that can handle input: <class 'util.Generators.garnetDataGenerator'>, <class 'NoneType'>

In [ ]:

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.yscale('log')
plt.legend(['train', 'test'], loc='upper right')
plt.show()
